STILL UNDER DEVELOPMENT (needs more description and different algorithms :P)

We work with a dataset containing 50,000 movie reviews from IMDB, labeled by sentiment (positive/negative).  In addition, there are another 50,000 IMDB reviews provided without any rating labels.  

The reviews are split evenly into train and test sets (25k train and 25k test). The overall distribution of labels is also balanced within the train and test sets (12.5k pos and 12.5k neg).  Our goal is to predict sentiment in the test dataset. 

In [1]:
import os                                # accessing directory of files
import pandas as pd                      # storing the data
from bs4 import BeautifulSoup            # removing HTML tags
import re                                # text processing with regular expressions
from gensim.models import word2vec       # embedding algorithm
import numpy as np                       # arrays and other mathy structures     
from tqdm import tqdm                    # timing algorithms
from gensim import models                # doc2vec implementation
from random import shuffle               # for shuffling reviews
from sklearn.linear_model import LogisticRegression
import nltk.data                         # sentence splitting
from keras.models import Sequential      # deep learning (part 1)
from keras.layers import Dense, Dropout  # deep learning (part 2)
from classes import Doc2VecUtility
from classes import Word2VecUtility
%matplotlib inline                       

# If you are using Python 3, you will get an error.
# (Pattern is a Python 2 library and fails to install for Python 3.)

/Users/alexis/anaconda/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
Using TensorFlow backend.


The dataset can be downloaded [here](http://ai.stanford.edu/~amaas/data/sentiment/).  We have written code to extract the reviews into Pandas dataframes.

In [2]:
def load_data(directory_name):
    # load dataset from directory to Pandas dataframe
    data = []
    files = [f for f in os.listdir('../../../aclImdb/' + directory_name)]
    for f in files:
        with open('../../../aclImdb/' + directory_name + f, "r", encoding = 'utf-8') as myfile:
            data.append(myfile.read())
    df = pd.DataFrame({'review': data, 'file': files})
    return df

# load training dataset
train_pos = load_data('train/pos/')
train_neg = load_data('train/neg/')

# load test dataset
test_pos = load_data('test/pos/')
test_neg = load_data('test/neg/')

# load unsupervised dataset
unsup = load_data('train/unsup/')

print("\n %d pos train reviews \n %d neg train reviews \n %d pos test reviews \n %d neg test reviews \n %d unsup reviews" \
      % (train_pos.shape[0], train_neg.shape[0], test_pos.shape[0], test_neg.shape[0], unsup.shape[0]))
print("\n TOTAL: %d reviews" % int(train_pos.shape[0] + train_neg.shape[0] + test_pos.shape[0] + test_neg.shape[0] + unsup.shape[0]))


 12500 pos train reviews 
 12500 neg train reviews 
 12500 pos test reviews 
 12500 neg test reviews 
 50000 unsup reviews

 TOTAL: 100000 reviews


`train_pos`, `train_neg`, `test_pos`, `test_neg`, and `unsup` are Pandas dataframes.  They each have two columns, and each row corresponds to a review:
- `file` : name of file that contains review
- `review` : the full text of the review

Each review is cleaned before being fed into a classification algorithm.
- HTML tags are removed through the use of the Beautiful Soup library.
- Punctuation is made consistent through the use of regular expressions.  This code appears below, in the `clean_str` method.
- All words are converted to lowercase.
- Each review is converted into either a list of words or to a list of lists of words.  In the latter, each list of lists corresponds to a sentence.

In [3]:
def clean_str( string ):
    # Function that cleans text using regular expressions
    string = re.sub(r' +', ' ', string)
    string = re.sub(r'\.+', '.', string)
    string = re.sub(r'\.(?! )', '. ', string)    
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'m", " \'m", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " ( ", string) 
    string = re.sub(r"\)", " ) ", string) 
    string = re.sub(r"\?", " ? ", string) 
    string = re.sub(r"\.", " . ", string)
    string = re.sub(r"\-", " - ", string)
    string = re.sub(r"\;", " ; ", string)
    string = re.sub(r"\:", " : ", string)
    string = re.sub(r'\"', ' " ', string)
    string = re.sub(r'\/', ' / ', string)
    return string

We note that there is still some room for improvement.  For instance, 
- Strings like "Sgt. Cutter" currently are broken into two sentences.  We should instead determine how to differentiate between periods that signify the end of an abbreviation and periods that denote the end of a sentence.
- Some writers separate their sentences with commas or line breaks; the algorithm currently absorbs these multiple sentences into an individual sentence.
- Ellipses (...) are currently processed as multiple, empty sentences (which are then discarded).

Before writing this post, I read the Kaggle tutorial [here](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-2-word-vectors).  My current processing algorithm borrows from that page, but also adds some meaningful improvements, partially informed by the algorithm [here](https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py).  For instance, 
- We keep the punctuation that ends each sentence (i.e., period vs. exclamation point), whereas punctuation was discarded in the Kaggle tutorial.  
- We do smarter processing of contractions, in a way that understands that "should've" = "should" + "'ve".  In the Kaggle tutorial, "should've" is kept as a single word (contractions are not understood in terms of their composite parts).

We can now see how well our Doc2Vec embedding performs.  To peek at the code under the hood, please look at the Python file [here](https://github.com/alexisbcook/Pelican_blog_all/blob/master/content/notebooks/classes/Doc2VecUtility.py).

In [4]:
[d2v_train, d2v_test] = Doc2VecUtility.get_embedding()

In [5]:
train_labels = np.append(np.ones(12500), np.zeros(12500))
test_labels = np.append(np.ones(12500), np.zeros(12500))

In [6]:
classifier = LogisticRegression()
classifier.fit(d2v_train, train_labels)
classifier.score(d2v_test, test_labels)

0.89456000000000002

Woohoo!  We can predict sentiment with nearly 90 percent accuracy!  Can we do better?  Let's try out a MLP with one hidden layer.

In [7]:
# Now, we try a multilayer perceptron (with one hidden layer) in Keras !
keras_model = Sequential()
keras_model.add(Dense(200, input_dim=300, init='uniform', activation='relu'))
keras_model.add(Dropout(0.5))
keras_model.add(Dense(1, activation='sigmoid'))
keras_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

keras_model.fit(d2v_train, train_labels, nb_epoch=10, batch_size=20)
loss, accuracy = keras_model.evaluate(d2v_test, test_labels, verbose=0)
print("\n", accuracy)

Epoch 1/10
25000/25000 [==============================] - 2s - loss: 0.2987 - acc: 0.8782     
Epoch 2/10
25000/25000 [==============================] - 2s - loss: 0.2624 - acc: 0.8904     
Epoch 3/10
25000/25000 [==============================] - 2s - loss: 0.2501 - acc: 0.8974     
Epoch 4/10
25000/25000 [==============================] - 2s - loss: 0.2370 - acc: 0.9046     
Epoch 5/10
25000/25000 [==============================] - 2s - loss: 0.2255 - acc: 0.9110     
Epoch 6/10
25000/25000 [==============================] - 2s - loss: 0.2130 - acc: 0.9187     
Epoch 7/10
25000/25000 [==============================] - 2s - loss: 0.1971 - acc: 0.9258     
Epoch 8/10
25000/25000 [==============================] - 2s - loss: 0.1843 - acc: 0.9316     
Epoch 9/10
25000/25000 [==============================] - 2s - loss: 0.1703 - acc: 0.9391     
Epoch 10/10
25000/25000 [==============================] - 2s - loss: 0.1578 - acc: 0.9437     

 0.9056


Hmm, it's a slight improvement, but we haven't done much better.  To improve, our intuition tells us we should try some combination of:
- better string cleaning,
- testing other parameters for Doc2Vec embedding step, and
- constructing deeper neural networks.

However, it seems that Kaggle competitiors were not able to make this work -- that is, we won't be able to do much better with the current plan.  Thus, we will try something else, informed by the techniques [here](https://www.kaggle.com/c/word2vec-nlp-tutorial/forums/t/14966/post-competition-solutions).  Namely, we will train a few more embedding models and then use an ensemble method.

We train a Word2Vec embedding, which we expect will get slightly worse performance than the Doc2Vec model.  To peek at the code under the hood, please look at the Python file [here](https://github.com/alexisbcook/Pelican_blog_all/blob/master/content/notebooks/classes/Word2VecUtility.py).

In [8]:
[w2v_train, w2v_test] = Word2VecUtility.get_embedding()

100%|██████████| 12500/12500 [00:13<00:00, 939.78it/s]


In [9]:
classifier = LogisticRegression()
classifier.fit(w2v_train, train_labels)
classifier.score(w2v_test, test_labels)

0.81396000000000002

In [10]:
# Now, we try a multilayer perceptron (with one hidden layer) in Keras !
keras_model = Sequential()
keras_model.add(Dense(200, input_dim=300, init='uniform', activation='relu'))
keras_model.add(Dropout(0.5))
keras_model.add(Dense(1, activation='sigmoid'))
keras_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

keras_model.fit(w2v_train, train_labels, nb_epoch=10, batch_size=20)
loss, accuracy = keras_model.evaluate(w2v_test, test_labels, verbose=0)
print("\n", accuracy)

Epoch 1/10
25000/25000 [==============================] - 2s - loss: 0.5246 - acc: 0.7510     
Epoch 2/10
25000/25000 [==============================] - 2s - loss: 0.3867 - acc: 0.8326     
Epoch 3/10
25000/25000 [==============================] - 2s - loss: 0.3584 - acc: 0.8472     
Epoch 4/10
25000/25000 [==============================] - 2s - loss: 0.3470 - acc: 0.8516     
Epoch 5/10
25000/25000 [==============================] - 2s - loss: 0.3395 - acc: 0.8562     
Epoch 6/10
25000/25000 [==============================] - 2s - loss: 0.3358 - acc: 0.8568     
Epoch 7/10
25000/25000 [==============================] - 2s - loss: 0.3318 - acc: 0.8595     
Epoch 8/10
25000/25000 [==============================] - 2s - loss: 0.3298 - acc: 0.8615     
Epoch 9/10
25000/25000 [==============================] - 2s - loss: 0.3282 - acc: 0.8622     
Epoch 10/10
25000/25000 [==============================] - 2s - loss: 0.3250 - acc: 0.8620     

 0.86376


As expected, we the Doc2Vec embedding provides better accuracy than the Word2Vec embedding (both with logistic regression and an MLP).  However, combining the two should result in an increase in performance, as we will see below.

In [11]:
# Combine Doc2Vec and Word2Vec features
d2v_w2v_train = np.append(d2v_train, w2v_train, axis=1)
d2v_w2v_test = np.append(d2v_test, w2v_test, axis=1)

In [12]:
classifier = LogisticRegression()
classifier.fit(d2v_w2v_train, train_labels)
classifier.score(d2v_w2v_test, test_labels)

0.89576

In [13]:
# Now, we try a multilayer perceptron (with one hidden layer) in Keras !
keras_model = Sequential()
keras_model.add(Dense(200, input_dim=600, init='uniform', activation='relu'))
keras_model.add(Dropout(0.5))
keras_model.add(Dense(1, activation='sigmoid'))
keras_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

keras_model.fit(d2v_w2v_train, train_labels, nb_epoch=10, batch_size=20)
loss, accuracy = keras_model.evaluate(d2v_w2v_test, test_labels, verbose=0)
print("\n", accuracy)

Epoch 1/10
25000/25000 [==============================] - 3s - loss: 0.2955 - acc: 0.8801     
Epoch 2/10
25000/25000 [==============================] - 3s - loss: 0.2602 - acc: 0.8928     
Epoch 3/10
25000/25000 [==============================] - 3s - loss: 0.2465 - acc: 0.8998     
Epoch 4/10
25000/25000 [==============================] - 3s - loss: 0.2359 - acc: 0.9051     
Epoch 5/10
25000/25000 [==============================] - 3s - loss: 0.2227 - acc: 0.9132     
Epoch 6/10
25000/25000 [==============================] - 3s - loss: 0.2104 - acc: 0.9197     
Epoch 7/10
25000/25000 [==============================] - 3s - loss: 0.1957 - acc: 0.9257     
Epoch 8/10
25000/25000 [==============================] - 3s - loss: 0.1817 - acc: 0.9340     
Epoch 9/10
25000/25000 [==============================] - 3s - loss: 0.1650 - acc: 0.9414     
Epoch 10/10
25000/25000 [==============================] - 3s - loss: 0.1531 - acc: 0.9469     

 0.905


Here, the performace is basically the same as when we used Doc2Vec alone.  I bet we can do better :)!  More on that soon!